<a href="https://colab.research.google.com/github/jacob-t-radford/ai-weather-notebooks/blob/main/Visualizing_Historical_AIWP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Accessing and visualizing historical AI Weather Prediction (AIWP) model output

This notebook will guide you through the process of accessing our historical AIWP reforecast archive and visualizing the output. If you're more interested in running AIWP models yourself, use [our other notebook](https://colab.research.google.com/github/jacob-t-radford/ai-weather-notebooks/blob/main/Running_AIWP.ipynb).

Estimated Time: 20 minutes

Questions? Contact jacob.radford@noaa.gov

#AI Weather Prediction Background Information

##What is an AI Weather Prediction Model?

AIWP models are weather models that have been trained to identify relationships between the state of the atmosphere at time t=0 and the state of the atmosphere at time t+1 using historical data like ECMWF's Reanalysis v5 (ERA5). The model can then be combined with any initial atmospheric state to predict a future state. For example, the model can use the atmospheric state at 00 UTC on Jan 1, 2025 to make a weather prediction for 06 UTC on Jan 1, 2025. It then takes its prediction for 06 UTC on Jan 1, 2025 as the input to predict the state at 12 UTC on Jan 1, 2025, and so on until the desired forecast length is reached (usually 10 days). This iterative process is referred to as an autoregressive model.

##How do they differ from traditional weather models (AKA Numerical Weather Prediction)?

Numerical Weather Prediction models use partial differential equations that represent our physical understanding of the atmosphere - things like conservation of momentum, conservation of energy, etc. AIWP models don't need any of this information because, in a way, they can learn these sorts of relationships from the historical data. NWP models are very computationally expensive and take a long time to run, even on supercomputers. Meanwhile, you can run an AIWP model right here!

##What about forecast skill? How do AIWP models compare to NWP models?

State of the art AIWP models have been shown to have performance comparable to or better than NWP models in many regards. For example, in this chart from WeatherBench2, blue squares indicate variables where the AIWP model outperforms the IFS HRES NWP model (a state of the art NWP model) in terms of root mean squared error (RMSE). The Pangu-Weather and GraphCast squares are almost all blue!

![WeatherBench2](https://shorturl.at/AUtBV)

Results have been very exciting for the meteorological community. That said, there are some caveats and perhaps the the biggest one is that which model is "better" can be subjective. Just because one model has lower RMSE than another doesn't mean it's necessarily better for your use case. For example, a couple of commonly cited limitations for AIWP models is that they may struggle to predict extreme events like strong hurricanes or intense precipitation and that they tend to produce "blurry" looking output for longer range forecasts. Furthermore, one model may perform better than another in particular regions or for specific types of weather events. More research is being done to identify specific strengths and weaknesses of different models.

##If I can run competitive weather models myself, will that make NOAA obsolete?

Definitely not. There are many reasons that NOAA will have a vital role in the future of weather forecasting, AI-based or otherwise:

1.   AIWP models still fundamentally rely on NOAA's data assimilation processes. Data assimilation is the process through which weather observations across the globe are combined to create a "best-guess" at the current state of the atmosphere. The output of the data assimilation system is what is then used to initialize AIWP models.
2.   Most AIWP models are limited in spatial resolution to 0.25° (~30km) because they are trained on ERA5 data, which itself has a resolution of 0.25°. 0.25° models are unable to capture many smaller, impactful meteorological features like thunderstorms. Traditional NWP models like the High-Resolution Rapid Refresh have spatial resolution of 3km and are much more capable of resolving smaller scale features.
3.   AIWP models predict a limited subset of meteorological variables. For example, variables related to precipitation type, wind gusts, and cloud parameters are currently not predicted by any AIWP models.
4.   There are still lingering questions about how well AIWP models can represent extreme weather events like hurricanes, intense precipitation, cold snaps, heat waves, etc.
5.   NOAA is taking an active role in AIWP model research and exploring how to incorporate AIWP into operations to make weather forecasts more accurate.
6.   The human factor: more accurate models are only one part of the equation. How the output of these models is then communicated to the general public is where NOAA really shines.


#Visualizing historical AIWP model output

##Connecting to a runtime

In the upper right corner of this page you will see "Connect" with a dropdown arrow next to it. This is how you will connect to cloud compute resources. For this purpose we just need a CPU.

**Click the dropdown arrow, then "change runtime type," select CPU and save. Finally, click "connect" to connect to the runtime.**

##Configuring the environment

In the first line we install all of the required packages.

You don't need to understand the details, but these packages will allow us to access just the pieces of our reforecast archive stored in an AWS S3 bucket that we need. This avoids having to download large netCDF files.

The second line here downloads some lightweight reference files. They are a map telling us where the data we need is within the S3 bucket and within the netCDF files.

In [ ]:
%%capture

#Install packages
!pip install -U --no-cache-dir fsspec kerchunk distributed virtualizarr pandas cartopy dask s3fs fastparquet awscli

#Download parquet files
!aws s3 cp --recursive s3://noaa-oar-mlwp-data/parquet/ . --no-sign-request

##Define the variable information

This cell is just a large dictionary defining some appropriate bounds for each variable and other general information like name and units.

**Only change this dictionary if you want to update minimum and maximum contour levels for a variable**

In [ ]:
# Don't modify this unless you want to update contour levels

variable_levels = {
    # Surface variables
    'msl': {'min': 97500, 'max': 102500, 'name': 'mean sea level pressure', 'units': 'Pa'},
    'u10': {'min': -25, 'max': 25, 'name': '10m U wind component', 'units': 'm/s'},
    'v10': {'min': -25, 'max': 25, 'name': '10m V wind component', 'units': 'm/s'},
    'u100': {'min': -30, 'max': 30, 'name': '100m U wind component', 'units': 'm/s'},
    'v100': {'min': -30, 'max': 30, 'name': '100m V wind component', 'units': 'm/s'},
    't2': {'min': 179, 'max': 321, 'name': '2m temperature', 'units': 'K'},
    'tcwv': {'min': 0, 'max': 70, 'name': 'total column water vapor', 'units': 'kg/m²'},
    'sp': {'min': 75000, 'max': 105000, 'name': 'surface pressure', 'units': 'Pa'},
    'tp': {'min': 0, 'max': 2, 'name': 'total precipitation', 'units': 'm'},

    # Geopotential height (z) variables
    'z50': {'min': 168000, 'max': 213000, 'name': 'geopotential at 50 hPa', 'units': 'm^2/s^2'},
    'z100': {'min': 133000, 'max': 166000, 'name': 'geopotential at 100 hPa', 'units': 'm^2/s^2'},
    'z150': {'min': 109000, 'max': 143000, 'name': 'geopotential at 150 hPa', 'units': 'm^2/s^2'},
    'z200': {'min': 92000, 'max': 124000, 'name': 'geopotential at 200 hPa', 'units': 'm^2/s^2'},
    'z250': {'min': 80000, 'max': 109000, 'name': 'geopotential at 250 hPa', 'units': 'm^2/s^2'},
    'z300': {'min': 69300, 'max': 96400, 'name': 'geopotential at 300 hPa', 'units': 'm^2/s^2'},
    'z400': {'min': 52600, 'max': 75700, 'name': 'geopotential at 400 hPa', 'units': 'm^2/s^2'},
    'z500': {'min': 39200, 'max': 58900, 'name': 'geopotential at 500 hPa', 'units': 'm^2/s^2'},
    'z600': {'min': 28400, 'max': 44600, 'name': 'geopotential at 600 hPa', 'units': 'm^2/s^2'},
    'z700': {'min': 19200, 'max': 32300, 'name': 'geopotential at 700 hPa', 'units': 'm^2/s^2'},
    'z850': {'min': 6640, 'max': 16400, 'name': 'geopotential at 850 hPa', 'units': 'm^2/s^2'},
    'z925': {'min': 1600, 'max': 9600, 'name': 'geopotential at 925 hPa', 'units': 'm^2/s^2'},
    'z1000': {'min': -4000, 'max': 4000, 'name': 'geopotential at 1000 hPa', 'units': 'm^2/s^2'},

    # Temperature (t) variables
    't50': {'min': 180, 'max': 240, 'name': 'temperature at 50 hPa', 'units': 'K'},
    't100': {'min': 187, 'max': 237, 'name': 'temperature at 100 hPa', 'units': 'K'},
    't150': {'min': 190, 'max': 240, 'name': 'temperature at 150 hPa', 'units': 'K'},
    't200': {'min': 193, 'max': 243, 'name': 'temperature at 200 hPa', 'units': 'K'},
    't250': {'min': 196, 'max': 243, 'name': 'temperature at 250 hPa', 'units': 'K'},
    't300': {'min': 196, 'max': 252, 'name': 'temperature at 300 hPa', 'units': 'K'},
    't400': {'min': 203, 'max': 266, 'name': 'temperature at 400 hPa', 'units': 'K'},
    't500': {'min': 209, 'max': 281, 'name': 'temperature at 500 hPa', 'units': 'K'},
    't600': {'min': 211, 'max': 293, 'name': 'temperature at 600 hPa', 'units': 'K'},
    't700': {'min': 202, 'max': 302, 'name': 'temperature at 700 hPa', 'units': 'K'},
    't850': {'min': 198, 'max': 313, 'name': 'temperature at 850 hPa', 'units': 'K'},
    't925': {'min': 201, 'max': 319, 'name': 'temperature at 925 hPa', 'units': 'K'},
    't1000': {'min': 204, 'max': 323, 'name': 'temperature at 1000 hPa', 'units': 'K'},

    # Wind (u and v) components
    'u50':{'min': -70, 'max': 70, 'name': 'U wind component at 50 hPa', 'units': 'm/s'},
    'u100':{'min': -70, 'max': 70, 'name': 'U wind component at 100 hPa', 'units': 'm/s'},
    'u150':{'min': -70, 'max': 70, 'name': 'U wind component at 150 hPa', 'units': 'm/s'},
    'u200':{'min': -70, 'max': 70, 'name': 'U wind component at 200 hPa', 'units': 'm/s'},
    'u250':{'min': -70, 'max': 70, 'name': 'U wind component at 250 hPa', 'units': 'm/s'},
    'u300':{'min': -70, 'max': 70, 'name': 'U wind component at 300 hPa', 'units': 'm/s'},
    'u400':{'min': -50, 'max': 50, 'name': 'U wind component at 400 hPa', 'units': 'm/s'},
    'u500':{'min': -50, 'max': 50, 'name': 'U wind component at 500 hPa', 'units': 'm/s'},
    'u600':{'min': -40, 'max': 40, 'name': 'U wind component at 600 hPa', 'units': 'm/s'},
    'u700':{'min': -40, 'max': 40, 'name': 'U wind component at 700 hPa', 'units': 'm/s'},
    'u850':{'min': -30, 'max': 30, 'name': 'U wind component at 850 hPa', 'units': 'm/s'},
    'u925':{'min': -30, 'max': 30, 'name': 'U wind component at 925 hPa', 'units': 'm/s'},
    'u1000':{'min': -30, 'max': 30, 'name': 'U wind component at 1000 hPa', 'units': 'm/s'},

    'v50':{'min': -70, 'max': 70, 'name': 'V wind component at 50 hPa', 'units': 'm/s'},
    'v100':{'min': -70, 'max': 70, 'name': 'V wind component at 100 hPa', 'units': 'm/s'},
    'v150':{'min': -70, 'max': 70, 'name': 'V wind component at 150 hPa', 'units': 'm/s'},
    'v200':{'min': -70, 'max': 70, 'name': 'V wind component at 200 hPa', 'units': 'm/s'},
    'v250':{'min': -70, 'max': 70, 'name': 'V wind component at 250 hPa', 'units': 'm/s'},
    'v300':{'min': -70, 'max': 70, 'name': 'V wind component at 300 hPa', 'units': 'm/s'},
    'v400':{'min': -50, 'max': 50, 'name': 'V wind component at 400 hPa', 'units': 'm/s'},
    'v500':{'min': -50, 'max': 50, 'name': 'V wind component at 500 hPa', 'units': 'm/s'},
    'v600':{'min': -40, 'max': 40, 'name': 'V wind component at 600 hPa', 'units': 'm/s'},
    'v700':{'min': -40, 'max': 40, 'name': 'V wind component at 700 hPa', 'units': 'm/s'},
    'v850':{'min': -30, 'max': 30, 'name': 'V wind component at 850 hPa', 'units': 'm/s'},
    'v925':{'min': -30, 'max': 30, 'name': 'V wind component at 925 hPa', 'units': 'm/s'},
    'v1000':{'min': -30, 'max': 30, 'name': 'V wind component at 1000 hPa', 'units': 'm/s'},

    #specific humidity (q)
    'q50':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 50 hPa', 'units': 'kg/kg'},
    'q100':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 100 hPa', 'units': 'kg/kg'},
    'q150':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 150 hPa', 'units': 'kg/kg'},
    'q200':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 200 hPa', 'units': 'kg/kg'},
    'q250':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 250 hPa', 'units': 'kg/kg'},
    'q300':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 300 hPa', 'units': 'kg/kg'},
    'q400':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 400 hPa', 'units': 'kg/kg'},
    'q500':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 500 hPa', 'units': 'kg/kg'},
    'q600':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 600 hPa', 'units': 'kg/kg'},
    'q700':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 700 hPa', 'units': 'kg/kg'},
    'q850':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 850 hPa', 'units': 'kg/kg'},
    'q925':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 925 hPa', 'units': 'kg/kg'},
    'q1000':{'min': 0, 'max': 0.032, 'name': 'specific humidity at 1000 hPa', 'units': 'kg/kg'},

    # relative Humidity (r)
    **{f'r{level}': {'min': 0, 'max': 100, 'name': f'relative humidity at {level} hPa', 'units': '%'}
       for level in [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]},

    # Vertical Velocity (w)
    **{f'w{level}': {'min': -1, 'max': 1, 'name': f'vertical velocity at {level} hPa', 'units': 'm/s'}
       for level in [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]}
}

##Accessing and visualizing the data

This is the primary code for visualizing data.

You need to specify these options:

*   year, month, day, hour: Ints defining date

*   models: Any of these (can specify multiple in a list)
           "GRAP_v100_GFS", "GRAP_v100_IFS",
           "FOUR_v200_GFS", "FOUR_v200_IFS",
           "PANG_v100_GFS", "PANG_v100_GFS",
           "AURO_v100_GFS", "AURO_v100_GFS"
*   variable:

<img src="https://i.imgur.com/eAHYuPb.png" alt="Variables" width="600">

*   level: 0 for single level or any of
1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50 for pressure levels
*   lead_time: Any multiple of 6 between 0 (hrs) and 240 (hrs)
*   lat_bounds/lon_bounds: Coordinates to subset the data. lon_bounds should be in range (-180,180)
*   plot_size: Make the plot bigger or smaller
*   num_levels: Number of contour levels




In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Date and time of model initialization
year = 2024
month = 7
day = 2
hour = 12

# List of models to compare
models = ["GRAP_v100_GFS", "GRAP_v100_IFS",
          "PANG_v100_GFS", "PANG_v100_IFS",
          "FOUR_v200_GFS", "FOUR_v200_IFS"]

# Variable and level selection
variable = "msl"
level = 0

# Forecast lead time (multiples of six from 0 to 240)
lead_time = 0

# Define latitude/longitude bounds for subsetting
lat_bounds = (-90, 90)
lon_bounds = (-180, 180)

# Controls figure size
plot_size = 5

# Number of contour levels
num_levels = 21

# Colormap (non-divergent)
colormap = "viridis"



#    *** Don't modify anything from here down ***



#Function to adjust from 0 to 360 lons to -180 to 180 lons
def shiftgrid(lons):
    return np.where(lons > 180, lons - 360, lons)

# Format date
date = f"{year}-{str(month).zfill(2)}-{str(day).zfill(2)}T{str(hour).zfill(2)}:00:00"

# Adjust variable name if pressure level is used
varlevel = variable if level == 0 else f"{variable}{level}"

# Define colormap levels
max_val = variable_levels[varlevel]["max"]
min_val = variable_levels[varlevel]["min"]
levels = np.linspace(min_val, max_val, num_levels)

#These are the variables that need diverging color map
if variable in ['u10','v10','u100','v100','u','v','w']:
  colormap = "bwr"
else:
  colormap = colormap

storage_options = {
    "remote_protocol": "s3",
    "remote_options": {"anon": True},
}
open_dataset_options = {"chunks": {}}

# Dictionary to store selected data for each model
selected_data_dict = {}

for model in models:
    # Load dataset
    ds = xr.open_dataset(
        f"{model}_combined_all.parq",
        engine="kerchunk",
        storage_options=storage_options,
        open_dataset_options=open_dataset_options,
    )

    # Rename 'time' to 'lead_time' and assign correct range
    ds = ds.rename({"time": "lead_time"})
    ds["lead_time"] = range(0, 241, 6)

    # Check if the variable has a level dimension before selecting it
    if "level" in ds[variable].dims:
        selected_data = ds[variable].sel(init_time=date, lead_time=lead_time, level=level)
    else:
        selected_data = ds[variable].sel(init_time=date, lead_time=lead_time)

    # Load into memory
    selected_data = selected_data.compute()

    # Convert longitudes using shiftgrid
    shifted_lons = shiftgrid(selected_data.longitude.values)

    # Sort the shifted data
    selected_data = xr.DataArray(selected_data, coords=[selected_data.latitude, shifted_lons], dims=["latitude", "longitude"])
    selected_data = selected_data.sortby("longitude")  # Sort after shifting

    # Subset data based on bounds
    selected_data = selected_data.sel(latitude=slice(lat_bounds[1], lat_bounds[0]),
                                    longitude=slice(lon_bounds[0], lon_bounds[1]))


    # Store processed data
    selected_data_dict[model] = selected_data

# Determine subplot layout with max 2 columns
num_models = len(models)
cols = min(2, num_models)
rows = -(-num_models // cols)

# Calculate aspect ratio of the geographic region
lat_extent = abs(lat_bounds[1] - lat_bounds[0])
lon_extent = abs(lon_bounds[1] - lon_bounds[0])
aspect_ratio = lon_extent / lat_extent

# Dynamically adjust the figure size based on geographic aspect ratio
base_size = plot_size
figure_width = base_size * cols * aspect_ratio * 0.9
figure_height = base_size * rows * 0.9

# Create figure and axes
fig, axes = plt.subplots(rows, cols, figsize=(figure_width, figure_height),
                         subplot_kw={"projection": ccrs.PlateCarree()})


# Flatten axes
axes = np.atleast_1d(axes).flatten()

# Iterate over models and plot
for ax, (model, selected_data) in zip(axes, selected_data_dict.items()):

    # Set plot extent
    ax.set_extent([lon_bounds[0], lon_bounds[1], lat_bounds[0], lat_bounds[1]], crs=ccrs.PlateCarree())

    # Add basemaps
    ax.add_feature(cfeature.BORDERS, linestyle="-", linewidth=0.8, edgecolor="black")
    ax.add_feature(cfeature.COASTLINE, linewidth=1.0)
    ax.add_feature(cfeature.STATES, linestyle=":", edgecolor="gray")

    lon, lat = np.meshgrid(selected_data.longitude, selected_data.latitude)

    # Plot contours
    c = ax.contourf(lon, lat, selected_data.values, transform=ccrs.PlateCarree(), cmap=colormap, levels=levels, extend="both")
    ax.set_title(f"{model} at {lead_time}h (Init: {date})")

# Remove unused axes
for i in range(num_models, len(axes)):
    fig.delaxes(axes[i])

# Adjust spacing
plt.subplots_adjust(bottom=0.2, top=0.90, left=0.05, right=0.95, wspace=-0.2, hspace=0.1)

# Add colorbar axis
cbar_ax = fig.add_axes([0.15, 0.07, .7, 0.05])
cbar = plt.colorbar(c, cax=cbar_ax, orientation="horizontal")
cbar.set_label(f"{variable} ({variable_levels[varlevel]['units']})")

plt.show()


#Conclusion

AIWP models will be powerful new weather forecasting tools, but we need more research on AIWP verification and behavior. This notebook simplifies exploration of AIWP model output for retrospective forecasts.

If you find anything interesting or have improvement suggestions, email me at jacob.radford@noaa.gov